In [3]:
import requests

# On demande la liste complète de tous les réseaux mondiaux
url = "https://api.citybik.es/v2/networks"
headers = {'User-Agent': 'Mozilla/5.0'}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()
    
    print(f"Analyse de {len(data['networks'])} réseaux...\n")
    
    trouve = False
    for network in data['networks']:
        # On cherche si la ville contient "Bordeaux"
        city_name = network['location']['city']
        
        if "Bordeaux" in city_name:
            print(f"✅ TROUVÉ !")
            print(f"Ville : {city_name}")
            print(f"Nom du réseau : {network['name']}")
            print(f"⚠️ NOUVEL ID À UTILISER : {network['id']}")
            print(f"Nouvelle URL : https://api.citybik.es/v2/networks/{network['id']}")
            trouve = True
            break
            
    if not trouve:
        print("❌ Bordeaux n'est plus listé sur CityBik.es pour le moment.")
        print("Il faut utiliser la source officielle (voir Solution 2 ci-dessous).")

except Exception as e:
    print(f"Erreur : {e}")

Analyse de 787 réseaux...

✅ TROUVÉ !
Ville : Bordeaux
Nom du réseau : V³
⚠️ NOUVEL ID À UTILISER : v3-bordeaux
Nouvelle URL : https://api.citybik.es/v2/networks/v3-bordeaux


In [23]:
import requests

url = "https://api.citybik.es/v2/networks/v3-bordeaux"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()

    recherche = "Quinconces" 
    
    print(f"🔍 Résultats détaillés pour '{recherche}' :")
    print("-" * 50)

    count = 0
    for station in data['network']['stations']:
        nom_station = station['name']
        
        if recherche.lower() in nom_station.lower():
            # 1. Récupération des données de base
            total_velos = station['free_bikes']
            places = station['empty_slots']
            
            # 2. Extraction des données cachées dans 'extra'
            # On utilise .get() pour éviter un crash si l'info est absente
            extra_data = station.get('extra', {})
            
            # CityBik.es stocke souvent les électriques sous 'ebikes'
            nb_elec = extra_data.get('ebikes', 0)
            
            # 3. Calcul des vélos classiques (Total - Électriques)
            # On s'assure que le résultat n'est pas négatif (sécurité)
            nb_classique = max(0, total_velos - nb_elec)

            # Affichage
            print(f"📍 {nom_station}")
            print(f"   ⚡ Vélos Électriques : {nb_elec}")
            print(f"   🚲 Vélos Classiques  : {nb_classique}")
            print(f"   🅿️ Places disponibles: {places}")
            print(f"   (Total vélos sur place : {total_velos})")
            print("-" * 50)
            count += 1

    if count == 0:
        print("Aucune station trouvée.")

except Exception as e:
    print(f"Une erreur est survenue : {e}")

🔍 Résultats détaillés pour 'Quinconces' :
--------------------------------------------------
📍 Quinconces
   ⚡ Vélos Électriques : 2
   🚲 Vélos Classiques  : 13
   🅿️ Places disponibles: 50
   (Total vélos sur place : 15)
--------------------------------------------------
